In [29]:
import pandas as pd

In [30]:
# inspection date is daily!
inspections = pd.read_csv("../data/HCC_Inspections.csv")

# this data is hourly!
weather = pd.read_csv("../data/Hourly_Weather.csv")

# metadata on weather data
observations = pd.read_csv("../data/Weather_Observations.csv")

# need to aggregate and average the weather data
weather_merged = pd.merge(weather, observations[['ObsID', 'Date']], on='ObsID', how='left')
weather_merged = weather_merged.dropna()
for col in ['Temperature', 'Humidity', 'Dew_Point', 'Wind_Speed', 'Wind_Gust', 'Pressure', 'Precip']:
    weather_merged[col] = weather_merged[col].astype(str).str.strip()
    weather_merged[col] = pd.to_numeric(weather_merged[col])
weather_merged.head()



,WeatherID,ObsID,StationID,Temperature,Humidity,Dew_Point,Wind_Direction,Wind_Speed,Wind_Gust,Pressure,Precip,Condition,Sunrise,Sunset,Daylight_Hours,Date
0,1,1,1,54,45,33,VAR,7,0,29.54,0.0,Fair,7:07:00,19:35:00,12:28:00,3/29/2016
1,2,2,1,52,47,32,NNW,3,0,29.56,0.0,Fair,7:07:00,19:35:00,12:28:00,3/29/2016
2,3,3,1,51,48,32,CALM,0,0,29.57,0.0,Fair,7:07:00,19:35:00,12:28:00,3/29/2016
3,4,4,1,48,56,33,CALM,0,0,29.57,0.0,Fair,7:07:00,19:35:00,12:28:00,3/29/2016
4,5,5,1,46,63,34,CALM,0,0,29.58,0.0,Fair,7:07:00,19:35:00,12:28:00,3/29/2016


In [31]:
aggregate_cols = ['Temperature', 'Humidity', 'Dew_Point', 'Wind_Speed', 'Wind_Gust', 'Pressure', 'Precip']


daily_weather = weather_merged.groupby(['Date', 'StationID'])[aggregate_cols].mean().reset_index()
daily_weather['Date'] = pd.to_datetime(daily_weather['Date'])
daily_weather['Date'] = daily_weather['Date'].dt.strftime('%m/%d/%Y')

weather_stations = pd.read_csv("../data/Weather_Stations.csv")
daily_weather = pd.merge(daily_weather, weather_stations[['StationID', 'Station_City']], on='StationID', how='left')
daily_weather.to_csv("../data/Daily_Weather.csv", index=False)

In [32]:
inspections['Date'] = pd.to_datetime(inspections['Date'])
inspections['Date'] = inspections['Date'].dt.strftime('%m/%d/%Y')

# gather hive and apiary information
hives = pd.read_csv("../data/Hive_Information.csv")
apiary_info = pd.read_csv("../data/Apiary_Information.csv")

hives = pd.merge(hives, apiary_info[['ApiaryID', 'Apiary', 'City']], on='ApiaryID', how='left')
inspections_merged = pd.merge(inspections, hives, on='HiveID', how='inner')
inspections_merged = inspections_merged.dropna()
inspections_merged.head()
inspections_merged = inspections_merged.rename(columns={'City': 'Station_City'})
inspections_merged.to_csv("../data/derived/Detailed_Inspections.csv", index=False)

In [33]:
inspections_with_temp = pd.merge(inspections_merged, daily_weather, on=['Date', 'Station_City'], how='left')
inspections_with_temp = inspections_with_temp.dropna()
inspections_with_temp['Healthy_Binary'] = inspections_with_temp['Healthy'].map({'Yes': 1, 'No': 0})
inspections_with_temp.to_csv("../data/derived/Inspections_with_Weather.csv", index=False)

In [36]:
inspections_with_temp.head()


,InspectionID,HiveID,Date,Brood,Bees,Queen,Food,Stressors,Space,Percent_Met,...,Station_City,StationID,Temperature,Humidity,Dew_Point,Wind_Speed,Wind_Gust,Pressure,Precip,Healthy_Binary
2,3,1,08/01/2016,1.0,1.0,1.0,1.0,1.0,1.0,100,...,Durham,2.0,80.208333,80.375000,73.250000,5.750000,0.000000,29.591250,0.000,1
3,4,1,08/08/2016,1.0,1.0,1.0,1.0,1.0,1.0,100,...,Durham,2.0,78.083333,82.625000,72.166667,6.458333,2.333333,29.548750,0.025,1
4,5,1,08/15/2016,1.0,1.0,1.0,1.0,0.0,1.0,83,...,Durham,2.0,84.363636,70.727273,73.272727,6.227273,0.000000,29.707273,0.000,0
5,6,1,08/22/2016,1.0,1.0,1.0,1.0,1.0,1.0,100,...,Durham,2.0,77.083333,59.625000,60.625000,3.125000,0.000000,29.634583,0.000,1
6,7,1,08/29/2016,1.0,1.0,1.0,1.0,1.0,1.0,100,...,Durham,2.0,78.541667,66.291667,65.083333,5.791667,0.000000,29.658333,0.000,1


In [37]:
inspections_with_temp.columns

Index(['InspectionID', 'HiveID', 'Date', 'Brood', 'Bees', 'Queen', 'Food',
       'Stressors', 'Space', 'Percent_Met', 'Healthy', 'Hive_Tag', 'ApiaryID',
       'Apiary', 'Station_City', 'StationID', 'Temperature', 'Humidity',
       'Dew_Point', 'Wind_Speed', 'Wind_Gust', 'Pressure', 'Precip',
       'Healthy_Binary'],
      dtype='object')

In [38]:
print(inspections_with_temp.groupby(['Healthy_Binary'])['Percent_Met'].describe())

                count        mean        std    min    25%    50%    75%  \
Healthy_Binary                                                             
0               468.0   63.034188  22.973845    0.0   50.0   67.0   83.0   
1               166.0  100.000000   0.000000  100.0  100.0  100.0  100.0   

                  max  
Healthy_Binary         
0                92.0  
1               100.0  
